# Debugging Performance Issues

_Brief abstract/introduction/motivation.  State what the chapter is about in 1-2 paragraphs._
_Then, have an introduction video:_

In [ ]:
from bookutils import YouTubeVideo
YouTubeVideo("w4u5gCgPlmg")

**Prerequisites**

* _Refer to earlier chapters as notebooks here, as here:_ [Earlier Chapter](Debugger.ipynb).

## Synopsis
<!-- Automatically generated. Do not edit. -->

To [use the code provided in this chapter](Importing.ipynb), write

```python
>>> from debuggingbook.PerformanceDebugger import <identifier>
```

and then make use of the following features.


_For those only interested in using the code in this chapter (without wanting to know how it works), give an example.  This will be copied to the beginning of the chapter (before the first section) as text with rendered input and output._



In [ ]:
import bookutils

In [ ]:
import Intro_Debugging

## Some Long-Running Function

In [ ]:
from ChangeCounter import ChangeCounter, debuggingbook_change_counter

In [ ]:
# with Timer() as t:
#     change_counter = debuggingbook_change_counter(ChangeCounter)

## Simple Profiling

In [ ]:
import cProfile

In [ ]:
# cProfile.run('debuggingbook_change_counter(ChangeCounter)')

Mining calls `diff` for every two versions. This is expensive.

## Alternative: Use a Tracer

In [ ]:
# ignore
from typing import Any, Optional, Type, Union, Dict, Tuple, List

In [ ]:
from Intro_Debugging import remove_html_markup

In [ ]:
from bookutils import print_content

In [ ]:
import inspect

In [ ]:
print_content(inspect.getsource(remove_html_markup), '.py',
              start_line_number=238)

In [ ]:
import Timer

In [ ]:
from types import FrameType

In [ ]:
from Tracer import Tracer

In [ ]:
class PerformanceTracer(Tracer):
    def __init__(self) -> None:
        super().__init__()
        self.reset_timer()
        self.hits: Dict[Tuple[str, int], int] = {}
        self.time: Dict[Tuple[str, int], float] = {}

    def reset_timer(self) -> None:
        self.timer = Timer.Timer()

    def __enter__(self) -> Any:
        super().__enter__()
        self.reset_timer()
        return self

    def traceit(self, frame: FrameType, event: str, arg: Any) -> None:
        t = self.timer.elapsed_time()
        key = (frame.f_code.co_name, frame.f_lineno)

        self.hits.setdefault(key, 0)
        self.time.setdefault(key, 0.0)
        self.hits[key] += 1
        self.time[key] += t

        self.reset_timer()

In [ ]:
with PerformanceTracer() as perf_tracer:
    for i in range(10000):
        s = remove_html_markup('<b>foo</b>')

In [ ]:
perf_tracer.hits

In [ ]:
perf_tracer.time

In [ ]:
import inspect

In [ ]:
from bookutils import print_content

## Collect

In [ ]:
from StatisticalDebugger import CoverageCollector, SpectrumDebugger

In [ ]:
class MetricCollector(CoverageCollector):
    def metric(self, event: Any) -> Optional[float]:
        return None

    def all_metrics(self, func: str) -> List[float]:
        return []

    def total(self, func: str) -> float:
        return sum(self.all_metrics(func))

    def maximum(self, func: str) -> float:
        return max(self.all_metrics(func))

In [ ]:
Location = Tuple[str, int]

In [ ]:
class TimeCollector(MetricCollector):
    def __init__(self) -> None:
        super().__init__()
        self.reset_timer()
        self.time: Dict[Location, float] = {}
        self.add_items_to_ignore([Timer.Timer, Timer.clock])

    def collect(self, frame: FrameType, event: str, arg: Any) -> None:
        t = self.timer.elapsed_time()
        super().collect(frame, event, arg)
        location = (frame.f_code.co_name, frame.f_lineno)

        self.time.setdefault(location, 0.0)
        self.time[location] += t

        self.reset_timer()

    def reset_timer(self) -> None:
        self.timer = Timer.Timer()

    def __enter__(self) -> Any:
        super().__enter__()
        self.reset_timer()
        return self

    def metric(self, location: Any) -> Optional[float]:
        if location in self.time:
            return self.time[location]
        else:
            return None
        
    def all_metrics(self, func: str) -> List[float]:
        return [time
                for (func_name, lineno), time in self.time.items()
                if func_name == func]

In [ ]:
with TimeCollector() as collector:
    for i in range(100):
        s = remove_html_markup('<b>foo</b>')

In [ ]:
for location, time in collector.time.items():
    print(location, time)

In [ ]:
collector.total('remove_html_markup')

## Visualize

In [ ]:
class MetricDebugger(SpectrumDebugger):
    def metric(self, location: Location) -> float:
        sum = 0.0
        for outcome in self.collectors:
            for collector in self.collectors[outcome]:
                assert isinstance(collector, MetricCollector)
                m = collector.metric(location)
                if m is not None:
                    sum += m  # type: ignore

        return sum
    
    def total(self, func_name: str) -> float:
        total = 0.0
        for outcome in self.collectors:
            for collector in self.collectors[outcome]:
                assert isinstance(collector, MetricCollector)
                total += sum(collector.all_metrics(func_name))

        return total
    
    def maximum(self, func_name: str) -> float:
        maximum = 0.0
        for outcome in self.collectors:
            for collector in self.collectors[outcome]:
                assert isinstance(collector, MetricCollector)
                maximum = max(maximum, 
                              max(collector.all_metrics(func_name)))

        return maximum
    
    def suspiciousness(self, location: Location) -> float:
        func_name, _ = location
        return self.metric(location) / self.total(func_name)
    
    def color(self, location: Location) -> str:
        func_name, _ = location
        hue = 240  # blue
        saturation = 100  # fully saturated
        darkness = self.metric(location) / self.maximum(func_name)
        lightness = 100 - darkness * 25
        return f"hsl({hue}, {saturation}%, {lightness}%)"
    
    def tooltip(self, location: Location) -> str:
        return f"{super().tooltip(location)} {self.metric(location)}"

In [ ]:
class PerformanceDebugger(MetricDebugger):
    def __init__(self, collector_class: Type, log: bool = False):
        assert issubclass(collector_class, MetricCollector)
        super().__init__(collector_class, log=log)

In [ ]:
with PerformanceDebugger(TimeCollector) as debugger:
    for i in range(100):
        s = remove_html_markup('<b>foo</b>')

In [ ]:
print(debugger)

In [ ]:
debugger

## Other Metrics

In [ ]:
class HitCollector(MetricCollector):
    def __init__(self) -> None:
        super().__init__()
        self.hits: Dict[Location, int] = {}

    def collect(self, frame: FrameType, event: str, arg: Any) -> None:
        super().collect(frame, event, arg)
        location = (frame.f_code.co_name, frame.f_lineno)

        self.hits.setdefault(location, 0)
        self.hits[location] += 1
    
    def metric(self, location: Location) -> Optional[int]:
        if location in self.hits:
            return self.hits[location]
        else:
            return None
        
    def all_metrics(self, func: str) -> List[float]:
        return [hits
                for (func_name, lineno), hits in self.hits.items()
                if func_name == func]

In [ ]:
with PerformanceDebugger(HitCollector) as debugger:
    for i in range(100):
        s = remove_html_markup('<b>foo</b>')

In [ ]:
debugger.total('remove_html_markup')

In [ ]:
print(debugger)

In [ ]:
debugger

## _Section 1_

\todo{Add}

## Synopsis

_For those only interested in using the code in this chapter (without wanting to know how it works), give an example.  This will be copied to the beginning of the chapter (before the first section) as text with rendered input and output._

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _assertions_ to check conditions at runtime](Assertions.ipynb)
* [reduce _failing inputs_ for efficient debugging](DeltaDebugger.ipynb)


## Background

_Cite relevant works in the literature and put them into context, as in:_

The idea of ensuring that each expansion in the grammar is used at least once goes back to Burkhardt \cite{Burkhardt1967}, to be later rediscovered by Paul Purdom \cite{Purdom1972}.

## Exercises

_Close the chapter with a few exercises such that people have things to do.  To make the solutions hidden (to be revealed by the user), have them start with_

```
**Solution.**
```

_Your solution can then extend up to the next title (i.e., any markdown cell starting with `#`)._

_Running `make metadata` will automatically add metadata to the cells such that the cells will be hidden by default, and can be uncovered by the user.  The button will be introduced above the solution._

### Exercise 1: _Title_

_Text of the exercise_

In [ ]:
# Some code that is part of the exercise
pass

_Some more text for the exercise_

**Solution.** _Some text for the solution_

In [ ]:
# Some code for the solution
2 + 2

_Some more text for the solution_

### Exercise 2: _Title_

_Text of the exercise_

**Solution.** _Solution for the exercise_